In [1]:
%pylab notebook
import datacube
from datetime import datetime, timedelta
import pandas as pd
import pyproj
import csv

dc = datacube.Datacube(app='nbart-fieldsites')

Populating the interactive namespace from numpy and matplotlib


In [2]:
fieldsites = pd.read_csv(
    'jrsrp_20170430.csv',
    usecols=[
        'obs_key',
        'obs_time',
        'longitude',
        'latitude',
    ])

In [24]:
def query_cube(fieldsite_key, fieldsite_time, lon, lat):
    target_day = datetime.strptime(fieldsite_time, '%Y-%m-%d')
    time_before = (target_day - timedelta(days=60)).strftime('%Y-%m-%d')
    time_after = (target_day + timedelta(days=60)).strftime('%Y-%m-%d')
    
    query = {'time': (time_before, time_after)}
    
    wgs_84 = pyproj.Proj(init='epsg:4326')
    aus_albers = pyproj.Proj(init='epsg:3577')

    x, y = pyproj.transform(wgs_84, aus_albers, lon, lat)
    
    y_max = y-30
    y_min = y+30
    x_max = x-30
    x_min = x+30
    query['x'] = (x_min, x_max)
    query['y'] = (y_max, y_min)
    query['crs'] = 'EPSG:3577'
        
    query['output_crs'] = 'EPSG:3577'
    query['resolution'] = -25, 25
    
    query['measurements'] = ['blue',
                          'green',
                          'red', 
                          'nir',
                          'swir1', 
                          'swir2'
                          ]
    
    deltas = {}
    products = dc.list_products()
    delta = None
    observation_date = None
    observation_product = None
    for product in products.name:        
        #if(product.startswith('ls') and product.endswith('albers') and (product not in ['ls5_ndvi_albers','ls7_ndvi_albers','ls8_ndvi_albers'])):
        if(product in ['ls5_nbart_albers','ls7_nbart_albers','ls8_nbart_albers']):        
            datasets = dc.find_datasets(product=product,**query)
            for dataset in datasets:
                if(delta == None):
                    delta  = (target_day - dataset.center_time)
                    observation_date = dataset.center_time
                    observation_product = product
                    continue
                current_delta = (target_day - dataset.center_time)
                if(abs(current_delta) < abs(delta)):
                    delta  = current_delta
                    observation_date = dataset.center_time
                    observation_product = product
    if(observation_date):
        time_before = (observation_date - timedelta(days=1)).strftime('%Y-%m-%d')
        time_after = (observation_date + timedelta(days=1)).strftime('%Y-%m-%d')
        query['time'] = (time_before, time_after)
        data = dc.load(product=observation_product, **query)

        point = data.sel(x=x,y=y,time=target_day, method='nearest')
        blue = point.blue.data
        green = point.green.data
        red = point.red.data
        nir = point.nir.data
        swir1 = point.swir1.data
        swir2 = point.swir2.data

        return fieldsite_key, fieldsite_time, lon, lat, observation_product, observation_date, delta, blue, green, red, nir, swir1, swir2
    else:
        print('could not find: ' + fieldsite_time)
        return None


In [32]:
fieldsite_tuples = [tuple(x) for x in fieldsites.values]
index = 0
observation_tuples = []
for fieldsite in fieldsite_tuples: 
    result = query_cube(*fieldsite)
    if(result):
        observation_tuples.append(result)
    index = index + 1
    if(index >= 500):
        break

could not find: 2017-02-12
could not find: 2017-04-05


In [33]:
df = pd.DataFrame(observation_tuples,
                  columns=[
                      'fieldsite_key',
                      'fieldsite_time',
                      'lon',
                      'lat',
                      'observation_product',
                      'observation_date',
                      'time_delta',
                      'blue',
                      'green',
                      'red',
                      'nir',
                      'swir1',
                      'swir2'
                  ])

In [34]:
df

,fieldsite_key,fieldsite_time,lon,lat,observation_product,observation_date,time_delta,blue,green,red,nir,swir1,swir2
0,15282706_02742904_20101216_000000,2010-12-16,152.827067,-27.429043,ls5_nbart_albers,2010-12-15 23:31:57.500,0 days 00:28:02.500000,183,359,329,2276,1174,354
1,12191981_03088325_20101013_000000,2010-10-13,121.919808,-30.883252,ls7_nbart_albers,2010-10-10 01:33:03.500,2 days 22:26:56.500000,-999,-999,-999,-999,-999,-999
2,14432646_02760430_20021030_000000,2002-10-30,144.326462,-27.604300,ls7_nbart_albers,2002-10-25 00:06:59.500,4 days 23:53:00.500000,519,905,1632,2285,3454,2916
3,13399362_02377054_20120608_000000,2012-06-08,133.993630,-23.770550,ls7_nbart_albers,2012-06-15 00:55:40.500,-8 days +23:04:19.500000,755,954,1160,1768,2462,1866
4,13577022_02114920_20160804_000000,2016-08-04,135.770220,-21.149200,ls8_nbart_albers,2016-07-29 00:54:41.500,5 days 23:05:18.500000,515,866,1159,2378,3046,2326
5,12456729_01785321_20120828_000000,2012-08-28,124.567289,-17.853211,ls7_nbart_albers,2012-09-04 01:38:16.000,-8 days +22:21:44,886,1167,1447,2149,2717,2248
6,13466259_02126378_20150527_000000,2015-05-27,134.662596,-21.263788,ls7_nbart_albers,2015-05-23 01:00:19.000,3 days 22:59:41,635,1187,2006,2853,3790,2935
7,15310119_02595226_20100630_130000,2010-06-30,153.101190,-25.952261,ls7_nbart_albers,2010-06-21 23:40:03.500,8 days 00:19:56.500000,-999,-999,7122,7422,3647,1984
8,13747621_01831000_20110922_000000,2011-09-22,137.476219,-18.310003,ls5_nbart_albers,2011-09-18 00:42:21.000,3 days 23:17:39,524,901,1159,2099,2670,1860
9,14846057_02824703_20161101_000000,2016-11-01,148.460570,-28.247030,ls8_nbart_albers,2016-10-02 00:01:18.000,29 days 23:58:42,830,987,975,2327,1670,1165
